<h1 style='color:black'> Aspects of Functional Programming </h1>

This exposition is adapted from Bloomberg ENG Training 'Aspects of Functional Programming'.

For a brief description of functional programming, the following is from [Wikipedia](https://en.wikipedia.org/wiki/Functional_programming):

In computer science, functional programming is a programming paradigm where programs are constructed by applying and composing functions. It is a declarative programming paradigm in which function definitions are trees of expressions that map values to other values, rather than a sequence of imperative statements which update the running state of the program.

In functional programming, functions are treated as first-class citizens, meaning that they can be bound to names (including local identifiers), passed as arguments, and returned from other functions, just as any other data type can. This allows programs to be written in a declarative and composable style, where small functions are combined in a modular manner.

Functional programming is sometimes treated as synonymous with **purely functional programming**, a subset of functional programming which treats all functions as deterministic mathematical functions, or pure functions. When a pure function is called with some given arguments, it will always return the same result, and cannot be affected by any mutable state or other side effects. This is in contrast with impure procedures, common in imperative programming, which can have side effects (such as modifying the program's state or taking input from a user). Proponents of purely functional programming claim that by restricting side effects, programs can have fewer bugs, be easier to debug and test, and be more suited to formal verification.

**(You should find time to know about generator comprehension sometime.)**

There are two ways to associate state (data) with functions. <font color='red'> **Closure** </font> is one way. Though in principle associating state or data with function may be the opposite of functional programming, the way of closure works shows python can accommodate the principle of 'functions are first-class citizens'

In [ ]:
'''
Functions are objects.
li = [foo, dir, help] # They can be in a list...
li[1]() # ...and hence can be called using an index (don't forget the parantheses!)
bar = foo # They can also have alias
actions = { 7: foo, 11: dir} # They can be put in a dict, which can fulfill the functionality of the switch statement.
'''

# Enclosing a function inside a fuction can be a way to factory functions
def foo(x): 
    mol = 422
    def bar(): # A function enclosed in a function is not available in outside scope...
        # Any data within the outside function scope (but not the global scope) 
        # and used in the inner function is enclosed in the closere (i.e. part of the closure)
        print x, mol  
        print 'bar'
    return bar # ... unless it is returned. Call foo will return a functions which will otherwise be GC'ed.

func1 = foo(42) # Note that func1 is *not* an alise of bar
func2 = foo(666)
# What actually happens...
# func1 and func2 now are associated to an object, respectively, while both these objects point to bar().
# The data (42 and 666) are stored in the aforementioned objects, which allows for functions to have different states.
# The relation of func1 and func2 to bar are like two instances of the a class.
# This mechanism of function associated with data is called closure.

# So the following will return False
print func1 is func2

# The closured data can be assesse by
func1.__closure__  # Note that no parantheses for func1



There are two key points of <font color='red'> **decorators**</font>.
<ul>
    <li> Decorators are closures or wrappers of functions.
    <li> Once wrapped, the wrapped function replace the original function for any call to the original.
</ul>

Definitely with decorators, performance is affected due to the overhang. Applications of decorators include **timing**, **logging** and **authetications**.

The following block also talks about the usage of `*args` and `**kwargs` (the names of the arguments do not necessarily have to be `args` and `kwargs`).

In [1]:
from functools import wraps

lower, upper = range(2)

def foo(x, y, acme=2):
    """
    x is an int, y is float, returns some stuff
    """
    print('foo')
    return 41

# Mechanism of decorators...
# Step 1: a wrapper of the original function - decorators (tracer in this case) can be put in a module for everyone to import.
def tracer(func):
    # Below is a parameterized decorator, that is, a decorator that takes an extra argument. See below on how it is achieved.
    @wraps(func)
    def ifunc(*args, **kwargs):
        # *args takes any number of positional arguments - produces a tuple in function definition (hence order is important).
        # **kwargs takes any number of keyword arguments - pruduces a dictionary. Keyword arguments should be behind positional arguments.
        print(args)
        print('calling', func.__name__)
        # Unpack the tuples and dictionaries - use of asterisks similar to dereference.
        rv = func(*args, **kwargs)
        print('exiting')
        return rv
    # Also need to transport the meta data of function name and docstring before returning. 
    # But the following two lines have been fulfilled by @wraps(func)
    '''
    ifunc.__name__ = func.__name__
    ifunc.__doc__ = func.__doc__
    '''
    
    return ifunc

# Step 2: bind it back to the original one
foo = tracer(foo)

foo(41, 42, acme=90)

# The true syntax of decorators - by default decorator takes the function immediately below as the first parameter.
'''
@tracer
def foo():
    print 'foo'
'''
    
# Now let's go back and talk about parametrized decorator - it can be achieved by wrapping up decorators.
# Take tracer as an example which allows it to take the cases of lower and upper to return different messages.
'''
def tracer(case):
    def idec(func):
        def ifunc(*args, **kwargs):
            print args
            print 'calling' if case == lower else 'CALLING'
            rv = func(*args, **kwargs)
            print 'exiting'
            return rv
        return ifunc
    return idec
'''
    

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-eef94285f4da>, line 9)

The other way to associate state(data) with functions is by <font color='red'> **generators** </font>. In python, any function with yield is a generator. When a generator is called, an object is generated with the code in it - the code is not called but rather in a standby state. 

A function with `yield` is different from an ordinary function, in that whenever a generator is called, it is executed from the **last `yield` it was left off, with all the local variables and state reinstated**; while an ordinary function is always executed from top to bottom, each time define new local variables and states.

Besides using `yield` in a function to make a generator, another way is to use __generator expression__, e.g. `gen = (x*x for x in range(5))`

One application of generator is when we do not want the whole iteration to be read into memory. Another one concerns __YieldManager__ decorator (find out more about this).

In [ ]:
# For example
def gen():
    yield 1
    yield 7
    yield 11
    yield 42
    
go = gen()
next(go) # or go.next(). Yields are similar to breakpoints in debug...

# ... until we hit the end and we get a StopIteration. So we can also do:
for item in gen():
    print item
    
# Another example: even number generator. (
# This is a function solution. To compare, a class solution must have __iter__ and next defined 
# (recall that the next method must raise StopIteration when appropriate.)
def gen(limit):
    val = 0
    while val < limit:
        yield val
        val += 2
        
# Another way to get a generator object besides yield - applying functions on a list

ps = [1,7,11,42]

def with_tax(p):
    return p * 1.2

a = map(with_tax, ps) # not efficient on large list
b = map(lambda p: p*1.2, ps) # lambda function
c = [ p * 1.2 for p in ps ] # list comprehension - still takes up a lot of memory by producing the whole list
d = [ p * 1.2 for p in ps if p > 10 ] # We can do filtering as well.
g = ( p * 1.2 for p in ps if p > 10 ) # It is not a tuple! We have created a generator object and we can calculate on the fly.